In [1]:
from folder import MySQL
import os
import csv
import glob
import MySQLdb
import scrapy
from selenium import webdriver
from scrapy.selector import Selector
from scrapy.http import Request
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from sqlalchemy import *
import numpy as np
import mysql.connector
import lxml
from lxml import etree as ElementTree
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import ast
from urllib.parse import urljoin
from rake_nltk import Rake
import datetime
from selenium.webdriver.chrome.options import Options
import urllib.request
from pathlib import Path
import random
import string
from selenium.webdriver.support.ui import Select

In [2]:
chrome_options = Options()
chrome_options.add_argument("download.default_directory=D:\video")
driver_search=webdriver.Chrome('C://Users/USER/Desktop/sougou/chromedriver',chrome_options=chrome_options)
driver_search.get('http://person.amac.org.cn/pages/registration/amac-publicity-report.html')
driver_parse=webdriver.Chrome('C://Users/USER/Desktop/sougou/chromedriver',chrome_options=chrome_options)
driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId=699085&ptiId=0')

In [3]:
n=0
sql=MySQL.MySQL('root','19940228083','fund_database')
sql.create('company_data','(机构名称 varchar(100),员工人数 varchar(100),基金从业资格 varchar(100),基金销售从业资格 varchar(100),其他从业资格 varchar(100),PEVC从业资格 varchar(100),合规风控负责人 varchar(100),基金投资顾问 varchar(100),基金经理 varchar(100),投资经理 varchar(100),投资总监 varchar(100),机构类别 varchar(100))')
sql.create('person_data','(职业类别 varchar(100),机构类别 varchar(100),姓名 varchar(100),性别 varchar(100),学历 varchar(100),证书编号 varchar(100),从业机构 varchar(100),从业资格类别 varchar(100),证书取得日期 varchar(100),证书有效截止日期 varchar(100),注册变更记录 varchar(100),诚信记录 varchar(100))')
df1=pd.DataFrame(columns=['机构名称', '员工人数','基金从业资格', '基金销售从业资格','其他从业资格','PE/VC从业资格','合规风控负责人','基金投资顾问','基金经理','投资经理','投资总监','机构类别'])
df2=pd.DataFrame(columns=['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'])
s1 = Select(driver_search.find_element_by_id('otcId'))
s2 = Select(driver_search.find_element_by_xpath('//select[@class="ui-pg-selbox"]'))
s2.select_by_index(5)
sleep(10)
while True:
    page=0
    sel = Selector(text=driver_search.page_source)
    pagenumber=int(sel.xpath('//span[@id="sp_1"]//text()').extract_first())
    while pagenumber>page:
        pageinfo=sel.xpath('//tr[@id=*]').extract()
        for companyinfo in pageinfo:
            company=lxml.html.fromstring(companyinfo)
            query=company.xpath('//a[@href="#"]//@onclick')[0]
            query=''.join(s for s in query if s.isdigit())
            companytable=company.xpath('//td//text()')[1:]
            companytype=sel.xpath('//select[@id="otcId"]//text()').extract()[n]
            companytable.append(companytype)
            del companytable[3]
            df1.loc[len(df1)]=companytable
            sql.insert('company_data',['机构名称', '员工人数','基金从业资格', '基金销售从业资格','其他从业资格','PEVC从业资格','合规风控负责人','基金投资顾问','基金经理','投资经理','投资总监','机构类别'],companytable)
            if companytable[2]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=0')
                jobtype='基金从业资格'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[3]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=2')
                jobtype='基金销售从业资格'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[4]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=3')
                jobtype='其他从业资格'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[5]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=4')
                jobtype='PEVC从业资格'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[6]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=6')
                jobtype='合规风控负责人'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[7]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=7')
                jobtype='基金投资顾问'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[8]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=9')
                jobtype='基金经理'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[9]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=10')
                jobtype='投资经理'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)
            if companytable[10]!='0':
                driver_parse.get('http://person.amac.org.cn/pages/registration/amac-publicity-finish.html?aoiId='+query+'&ptiId=11')
                jobtype='投资总监'
                while True:
                    last_height = driver_parse.execute_script("return document.body.scrollHeight")
                    while True:
                        driver_parse.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        sleep(2)
                        new_height = driver_parse.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height
                    break
                table=Selector(text=driver_parse.page_source)
                personinfo=table.xpath('//body//tr').extract()[2:]
                for person in personinfo:
                    tr=lxml.html.fromstring(person)
                    persontable=[jobtype,companytable[-1]]
                    persontable.extend(tr.xpath('//td//text()'))
                    df2.loc[len(df2)]=persontable
                    sql.insert('person_data',['职业类别', '机构类别','姓名', '性别','学历','证书编号','从业机构','从业资格类别','证书取得日期','证书有效截止日期','注册变更记录','诚信记录'],persontable)            
        next_page = driver_search.find_element_by_xpath('//span[@class="ui-icon ui-icon-seek-next"]')
        next_page.click()
        page=page+1
        sleep(10)
        sel = Selector(text=driver_search.page_source)
    n=n+1
    s1.select_by_index(n)
    sleep(10)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=66.0.3359.139)
  (Driver info: chromedriver=2.37.544315 (730aa6a5fdba159ac9f4c1e8cbc59bf1b5ce12b7),platform=Windows NT 10.0.16299 x86_64)
